In [38]:
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import random
from torchvision.io import read_image
from torchvision import transforms
from torchvision import models
import os

## to get pretrained model from timm
import timm

In [39]:
lr = 1e-3
batch_size = 64

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

trans = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.491, 0.482, 0.447), (0.247, 0.243, 0.262))
])

root = '' ## 이미지 경로

original_trainset = datasets.ImageFolder(root=root+"/origin-cifar-10/train", transform=trans)
original_testset = datasets.ImageFolder(root=root+"origin-cifar-10/test", transform=trans)
svd_trainset = datasets.ImageFolder(root=root+"/svd-cifar-10/train", transform=trans)
svd_testset = datasets.ImageFolder(root=root+"/svd-cifar-10/test", transform=trans)
svd4_trainset = datasets.ImageFolder(root=root+"/svd4-cifar-10/train", transform=trans)
svd4_testset = datasets.ImageFolder(root=root+"/svd4-cifar-10/test", transform=trans)

o_trainloader = DataLoader(original_trainset, batch_size=batch_size, shuffle=True, drop_last=True)
o_testloader = DataLoader(original_testset, batch_size=batch_size, shuffle=False, drop_last=False)
s_trainloader = DataLoader(svd_trainset, batch_size=batch_size, shuffle=True, drop_last=True)
s_testloader = DataLoader(svd_testset, batch_size=batch_size, shuffle=False, drop_last=False)
s4_trainloader = DataLoader(svd4_trainset, batch_size=batch_size, shuffle=True, drop_last=True)
s4_testloader = DataLoader(svd4_testset, batch_size=batch_size, shuffle=False, drop_last=False)

classes = original_testset.classes

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

model1 = timm.create_model('xception', pretrained=True)
model2 = timm.create_model('xception', pretrained=True)
model3 = timm.create_model('xception', pretrained=True)

model1.fc = nn.Linear(in_features=model1.fc.in_features, out_features=10, bias=True)
model2.fc = nn.Linear(in_features=model2.fc.in_features, out_features=10, bias=True)
model3.fc = nn.Linear(in_features=model3.fc.in_features, out_features=10, bias=True)

nn.init.xavier_normal_(model1.fc.weight)
nn.init.xavier_normal_(model2.fc.weight)
nn.init.xavier_normal_(model3.fc.weight)

model1.to(device)
model2.to(device)
model3.to(device)

Using cuda device


Xception(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act2): ReLU(inplace=True)
  (block1): Block(
    (skip): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
    (skipbn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (rep): Sequential(
      (0): SeparableConv2d(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
        (pointwise): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): SeparableConv2d(
        (conv1): Conv

In [40]:
criterion = nn.CrossEntropyLoss()
optimizer1 = optim.Adam(model1.parameters(), lr=lr)
optimizer2 = optim.Adam(model2.parameters(), lr=lr)
optimizer3 = optim.Adam(model3.parameters(), lr=lr)

In [41]:
def train_loop(dataloader, model, criterion, optimizer):
    size = len(dataloader.dataset)
    loss_sum = 0
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X.to(device))
        loss = criterion(pred, y.to(device))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            loss_sum += loss
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

    return loss_sum / size

def test_loop(dataloader, model, criterion):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    hit, loss = 0, 0
    conf_matrix = []
    for i in range(10):
        conf_matrix.append([0.0, 0.0, 0.0, 0.0])

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X.to(device))
            loss += criterion(pred, y.to(device)).item()
            hit += (pred.argmax(1) == y.to(device)).type(torch.float).sum().item()
            temp = pred.argmax(1)
            for k in range(len(temp)):
                for i in range(10):
                    if i == temp[k]:
                        if i == y[k]:
                            conf_matrix[i][0] += 1.0
                        else:
                            conf_matrix[i][1] += 1.0
                    else:
                        if i == y[k]:
                            conf_matrix[i][2] += 1.0
                        else:
                            conf_matrix[i][3] += 1.0

    
    loss /= (size/batch_size)
    hit /= size
    print(f"Test Error: \n Accuracy: {(100*hit):>0.1f}%, Avg loss: {loss:>8f}\n")

    return conf_matrix

In [42]:
epochs = 20
train_loss1, test_res1, test_res2, test_res3 = [], [], [], []
for i in range(epochs):
    print(f"Epoch {i + 1}-------------------------------")
    model1.train()
    train_loss1.append(train_loop(o_trainloader, model1, criterion, optimizer1))
    model1.eval()
    test_res1.append(test_loop(o_testloader, model1, criterion))
    test_res2.append(test_loop(s_testloader, model1, criterion))
    test_res3.append(test_loop(s4_testloader, model1, criterion))

Epoch 1-------------------------------
loss: 2.313750 [    0/50000]
loss: 2.320264 [ 6400/50000]
loss: 1.600534 [12800/50000]
loss: 1.278136 [19200/50000]
loss: 1.041227 [25600/50000]
loss: 0.931858 [32000/50000]
loss: 0.874596 [38400/50000]
loss: 0.925266 [44800/50000]
Test Error: 
 Accuracy: 70.5%, Avg loss: 0.861171

Test Error: 
 Accuracy: 64.6%, Avg loss: 1.008877

Test Error: 
 Accuracy: 52.3%, Avg loss: 1.375887

Epoch 2-------------------------------
loss: 0.888448 [    0/50000]
loss: 0.824346 [ 6400/50000]
loss: 0.574079 [12800/50000]
loss: 0.880633 [19200/50000]
loss: 0.987906 [25600/50000]
loss: 0.740143 [32000/50000]
loss: 0.591463 [38400/50000]
loss: 0.636473 [44800/50000]
Test Error: 
 Accuracy: 78.6%, Avg loss: 0.636795

Test Error: 
 Accuracy: 73.9%, Avg loss: 0.758818

Test Error: 
 Accuracy: 58.8%, Avg loss: 1.190953

Epoch 3-------------------------------
loss: 0.433199 [    0/50000]
loss: 0.573900 [ 6400/50000]
loss: 0.572995 [12800/50000]
loss: 0.499318 [19200/5000

In [43]:
epochs = 20
train_loss2, test_res4, test_res5, test_res6 = [], [], [], []
for i in range(epochs):
    print(f"Epoch {i + 1}-------------------------------")
    model2.train()
    train_loss2.append(train_loop(s_trainloader, model2, criterion, optimizer2))
    model2.eval()
    test_res4.append(test_loop(o_testloader, model2, criterion))
    test_res5.append(test_loop(s_testloader, model2, criterion))
    test_res6.append(test_loop(s4_testloader, model2, criterion))

Epoch 1-------------------------------
loss: 2.310410 [    0/50000]
loss: 1.611678 [ 6400/50000]
loss: 1.601985 [12800/50000]
loss: 1.052712 [19200/50000]
loss: 0.879269 [25600/50000]
loss: 0.938844 [32000/50000]
loss: 0.901041 [38400/50000]
loss: 0.666927 [44800/50000]
Test Error: 
 Accuracy: 75.3%, Avg loss: 0.737142

Test Error: 
 Accuracy: 73.8%, Avg loss: 0.775327

Test Error: 
 Accuracy: 61.3%, Avg loss: 1.129648

Epoch 2-------------------------------
loss: 0.692542 [    0/50000]
loss: 0.639382 [ 6400/50000]
loss: 0.675053 [12800/50000]
loss: 0.859534 [19200/50000]
loss: 0.653885 [25600/50000]
loss: 0.733180 [32000/50000]
loss: 0.586695 [38400/50000]
loss: 0.571444 [44800/50000]
Test Error: 
 Accuracy: 79.0%, Avg loss: 0.619800

Test Error: 
 Accuracy: 76.4%, Avg loss: 0.688778

Test Error: 
 Accuracy: 63.0%, Avg loss: 1.100964

Epoch 3-------------------------------
loss: 0.394619 [    0/50000]
loss: 0.503122 [ 6400/50000]
loss: 0.361733 [12800/50000]
loss: 0.440063 [19200/5000

In [44]:
epochs = 20
train_loss3, test_res7, test_res8, test_res9 = [], [], [], []
for i in range(epochs):
    print(f"Epoch {i + 1}-------------------------------")
    model3.train()
    train_loss3.append(train_loop(s4_trainloader, model3, criterion, optimizer3))
    model3.eval()
    test_res7.append(test_loop(o_testloader, model3, criterion))
    test_res8.append(test_loop(s_testloader, model3, criterion))
    test_res9.append(test_loop(s4_testloader, model3, criterion))

Epoch 1-------------------------------
loss: 2.302337 [    0/50000]
loss: 1.886896 [ 6400/50000]
loss: 1.598097 [12800/50000]
loss: 1.359783 [19200/50000]
loss: 1.499762 [25600/50000]
loss: 0.965327 [32000/50000]
loss: 1.586606 [38400/50000]
loss: 0.955674 [44800/50000]
Test Error: 
 Accuracy: 62.0%, Avg loss: 1.080647

Test Error: 
 Accuracy: 64.2%, Avg loss: 1.012366

Test Error: 
 Accuracy: 65.0%, Avg loss: 0.982286

Epoch 2-------------------------------
loss: 0.911965 [    0/50000]
loss: 1.019206 [ 6400/50000]
loss: 0.609681 [12800/50000]
loss: 0.919861 [19200/50000]
loss: 0.800453 [25600/50000]
loss: 0.878897 [32000/50000]
loss: 0.762585 [38400/50000]
loss: 0.801115 [44800/50000]
Test Error: 
 Accuracy: 68.5%, Avg loss: 0.921927

Test Error: 
 Accuracy: 69.9%, Avg loss: 0.873085

Test Error: 
 Accuracy: 68.8%, Avg loss: 0.897551

Epoch 3-------------------------------
loss: 0.852983 [    0/50000]
loss: 0.694052 [ 6400/50000]
loss: 0.683400 [12800/50000]
loss: 0.760138 [19200/5000

In [45]:
print("Result for Testing with original datasets")
print(max(test_res1), max(test_res2), max(test_res3))

print("Result for Testing with original datasets")
print(max(test_res4), max(test_res5), max(test_res6))

print("Result for Testing with original datasets")
print(max(test_res7), max(test_res8), max(test_res9))

Result for Testing with original datasets
[[891.0, 286.0, 109.0, 8714.0], [918.0, 126.0, 82.0, 8874.0], [702.0, 192.0, 298.0, 8808.0], [624.0, 305.0, 376.0, 8695.0], [747.0, 143.0, 253.0, 8857.0], [685.0, 197.0, 315.0, 8803.0], [882.0, 182.0, 118.0, 8818.0], [901.0, 254.0, 99.0, 8746.0], [900.0, 97.0, 100.0, 8903.0], [862.0, 106.0, 138.0, 8894.0]] [[858.0, 238.0, 142.0, 8762.0], [846.0, 83.0, 154.0, 8917.0], [691.0, 232.0, 309.0, 8768.0], [649.0, 439.0, 351.0, 8561.0], [691.0, 174.0, 309.0, 8826.0], [668.0, 262.0, 332.0, 8738.0], [769.0, 132.0, 231.0, 8868.0], [880.0, 255.0, 120.0, 8745.0], [913.0, 153.0, 87.0, 8847.0], [871.0, 196.0, 129.0, 8804.0]] [[665.0, 302.0, 335.0, 8698.0], [326.0, 13.0, 674.0, 8987.0], [472.0, 222.0, 528.0, 8778.0], [676.0, 1016.0, 324.0, 7984.0], [749.0, 678.0, 251.0, 8322.0], [554.0, 462.0, 446.0, 8538.0], [280.0, 24.0, 720.0, 8976.0], [609.0, 94.0, 391.0, 8906.0], [868.0, 555.0, 132.0, 8445.0], [786.0, 649.0, 214.0, 8351.0]]
Result for Testing with original

In [49]:
def best_acc_finder(test_res):
    best, max = 0, 0
    for i in range(len(test_res)):
        correct = 0.0
        total = 0.0
        for j in range(len(test_res[i])):
            correct = test_res[i][j][0] +  test_res[i][j][3]
            total = correct + test_res[i][j][2] + test_res[i][j][1]
            if max < (correct / total):
                max = correct / total
                best = i

    print(f"best accuary: {max:>0.2f} at {best}")
    return best

In [50]:
best_acc_finder(test_res1)
best_acc_finder(test_res2)
best_acc_finder(test_res3)
best_acc_finder(test_res4)
best_acc_finder(test_res5)
best_acc_finder(test_res6)
best_acc_finder(test_res7)
best_acc_finder(test_res8)
best_acc_finder(test_res9)

best accuary: 0.98 at 8
best accuary: 0.98 at 11
best accuary: 0.96 at 13
best accuary: 0.98 at 15
best accuary: 0.98 at 7
best accuary: 0.97 at 4
best accuary: 0.98 at 8
best accuary: 0.98 at 8
best accuary: 0.97 at 11


11

In [53]:
def metrics_single_class(conf_mat):
    accuracy = (conf_mat[0] + conf_mat[3]) / (conf_mat[0] + conf_mat[1] + conf_mat[2] + conf_mat[3])
    precision = conf_mat[0] / (conf_mat[0] + conf_mat[1])
    recall = conf_mat[0] / (conf_mat[0] + conf_mat[2])
    f1_score = 2 * precision * recall / (precision + recall)
    
    return accuracy, precision, recall, f1_score

def metrics(conf_mats):
    sum_acc, sum_pre, sum_recall, sum_f1 = 0.0, 0.0, 0.0, 0.0
    for i in range(len(conf_mats)):
        temp_acc, temp_pre, temp_recall, temp_f1 = metrics_single_class(conf_mats[i])
        #print(f"For class-{i}:\n Accuracy: {temp_acc:>0.4f}, Precision: {temp_pre:>0.4f}, Recall: {temp_recall:>0.4f}, F1-score: {temp_f1:>0.4f}")
        sum_acc += temp_acc
        sum_pre += temp_pre
        sum_recall += temp_recall
        sum_f1 += temp_f1

    sum_acc /= 10
    sum_pre /= 10
    sum_recall /= 10
    sum_f1 /= 10

    return sum_acc, sum_pre, sum_recall, sum_f1


print(metrics(test_res1[best_acc_finder(test_res1)]))
print(metrics(test_res2[best_acc_finder(test_res2)]))
print(metrics(test_res3[best_acc_finder(test_res3)]))
print(metrics(test_res4[best_acc_finder(test_res4)]))
print(metrics(test_res5[best_acc_finder(test_res5)]))
print(metrics(test_res6[best_acc_finder(test_res6)]))
print(metrics(test_res7[best_acc_finder(test_res7)]))
print(metrics(test_res8[best_acc_finder(test_res8)]))
print(metrics(test_res9[best_acc_finder(test_res9)]))

best accuary: 0.98 at 8
(0.96434, 0.8235704590791609, 0.8217000000000001, 0.8216989353969355)
best accuary: 0.98 at 11
(0.9539, 0.7899591820907047, 0.7695, 0.7711250441640782)
best accuary: 0.96 at 13
(0.9211200000000002, 0.6748912726487852, 0.6055999999999999, 0.6020618723751362)
best accuary: 0.98 at 15
(0.9614999999999998, 0.8093928131522722, 0.8075000000000001, 0.8072381684426938)
best accuary: 0.98 at 7
(0.9603000000000002, 0.808426570019179, 0.8014999999999999, 0.8030111621887046)
best accuary: 0.97 at 4
(0.93956, 0.7297270599754124, 0.6977999999999999, 0.700065930162623)
best accuary: 0.98 at 8
(0.9511400000000002, 0.766867454860588, 0.7557, 0.7497770356513755)
best accuary: 0.98 at 8
(0.9518999999999999, 0.7650450892615127, 0.7595000000000001, 0.7555173730599272)
best accuary: 0.97 at 11
(0.9489000000000001, 0.7480693854228467, 0.7444999999999999, 0.7449372449749209)
